<a href="https://colab.research.google.com/github/bhadaur1/Geron/blob/master/Geron_Chap_10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook deals with wide and deep formulation of Keras model using California Housing dataset

### Just the deep model for now

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


In [3]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)


Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.7332 - val_loss: 0.5228
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5173 - val_loss: 0.4719
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4819 - val_loss: 0.4463
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4727 - val_loss: 0.4345
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4486 - val_loss: 0.4278
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4484 - val_loss: 0.4217
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4304 - val_loss: 0.4156
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4275 - val_loss: 0.4160
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4209 - val_loss: 0.4089
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4170 - val_loss: 0.4041

In [5]:
mse_test, y_pred

(0.3797622323036194, array([[2.9087143],
        [2.9145808],
        [1.6272081]], dtype=float32))

Creating wide and deep model using Keras functional API

In [8]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [13]:
def get_wide_and_deep_model():
  keras.backend.clear_session()
  input_ = keras.layers.Input(shape=X_train.shape[1:])
  hidden1 = keras.layers.Dense(30, activation="relu")(input_)
  hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
  concat = keras.layers.Concatenate()([input_, hidden2])
  output = keras.layers.Dense(1)(concat)
  model = keras.Model(inputs=[input_], outputs = [output])
  return model

model =get_wide_and_deep_model()

In [14]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_1[0][0]         

In [20]:
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)


Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.3000 - val_loss: 0.6532
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 0.6503 - val_loss: 0.4813
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.8770 - val_loss: 0.4546
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4485 - val_loss: 0.4348
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4345 - val_loss: 0.4292
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4528 - val_loss: 0.4173
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4149 - val_loss: 0.4041
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4090 - val_loss: 0.4001
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4338 - val_loss: 0.3919
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3922 - val_loss: 0.3864

In [21]:
mse_test, y_pred

(0.3478039801120758, array([[2.9203858],
        [2.7344778],
        [1.4872606]], dtype=float32))

In [24]:
def multi_input_model_training():

  keras.backend.clear_session()
  input_A = keras.layers.Input(shape=[5], name="wide_input")
  input_B = keras.layers.Input(shape=[6], name="deep_input")
  hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
  hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
  concat = keras.layers.concatenate([input_A, hidden2])
  output = keras.layers.Dense(1, name="output")(concat)
  model = keras.Model(inputs=[input_A, input_B], outputs=[output])

  model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))



  X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
  X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
  X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
  X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

  history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                      validation_data=((X_valid_A, X_valid_B), y_valid))
  mse_test = model.evaluate((X_test_A, X_test_B), y_test)
  y_pred = model.predict((X_new_A, X_new_B))

  return model, history, mse_test, y_pred

model, history, mse_test, y_pred = multi_input_model_training()

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.9751 - val_loss: 0.8400
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 0.7504 - val_loss: 0.6734
Epoch 3/20
363/363 [==============================] - 1s 1ms/step - loss: 0.6534 - val_loss: 0.6146
Epoch 4/20
363/363 [==============================] - 1s 1ms/step - loss: 0.6099 - val_loss: 0.5813
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5811 - val_loss: 0.5560
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5588 - val_loss: 0.5375
Epoch 7/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5416 - val_loss: 0.5219
Epoch 8/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5283 - val_loss: 0.5104
Epoch 9/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5178 - val_loss: 0.5022
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5095 - val_loss: 0.4946

In [25]:
mse_test, y_pred

(0.4760817289352417, array([[2.7399888],
        [3.2710438],
        [2.0303683]], dtype=float32))